# Chap 06 파이썬 고급 주제

# 6.1 멀티 프로세스와 멀티 스레드
<hr>

프로그램의 작업을 동시에 수행하는 방법은 멀티 프로세스와 멀티 스레드이다.

- **멀티 프로세스**
    * 별도의 메모리 영역을 가지며, 특별한 메커니즘으로만 통신 가능
    * 프로세스 간 데이터 공유와 통신용으로는 비효율적
    * 파이썬에서는 subprocess 모듈을 사용
    
- **멀티 스레드**
    * 단일 프로세스 내의 멀티 스레드는 동일한 메모리에 접근
    * threading 모듈의 처리를 ㅗㅇ해 한번에 한 스레드만 메모리 영역에 접근 가능
    * 각 프로세스가 독립적인 stack, heap, code, data 영역을 가지는 반면, 한 프로세스에 속한 스레드는 stack 영역을 제외한 메모리 영역을 공유
    
파이썬에 스레드 매커니즘이 있긴 하나, 진정한 병렬 실행이 지원되는 것은 아님. 하지만 오늘날 운영체제에서 충분히 효율적임.

## 6.1.1 subprocess 모듈

- subprocess 모듈은 '부모-자식' 프로세스 쌍을 생성하는 데 사용
- 부모 프로세스는 차례로 다른 일을 처리하는 자식 프로세스의 인스턴스를 실행
- 자식 프로세스를 사용함으로써 멀티 코어의 이점을 최대한 취하고, 동시성(concurrency) 문제를 운영체제가 알아서 처리하도록 한다

## 6.1.2 threading 모듈

- 스레드가 여러개로 분리되면, 스레드 간 데이터 고융의 복잡성이 증가
- 또한 lock과 deadlock을 회피하는 데 주의를 기울여야 함
- 파이썬에서는 단 하나의 메인 스레드만 존재
- 멀티 스레드를 사용하려면 threading 모듈 사용

- lock 관리를 위해 queue 모듈 사용
- 큐에 의존하면 자원의 접근을 직렬화할 수 있고, 이는 곧 한번에 하나의 스레드만 데이터에 접근할 수 있게 한다는 뜻(FIFO)

- worker thread가 작업을 완료했는데도 프로그램이 종료되지 않고 계속 실행되는 경우 문제가 될 수 있다
- 스레드를 데몬으로 변환하면 데몬 스레드가 실행되지 않는 즉시 프로그램이 종료
- queue.join() 메서드는 큐가 빌 때까지(큐의 모든 항목이 처리될 때까지) 기다린다

In [ ]:
import queue
import threading

q = queue.Queue()

def worker(num):
    while True:
        item = q.get()
        if item is None:
            break
        # 작업을 처리한다.
        print("스레드 {0} : 처리 완료 {1}".format(num+1, item))
        q.task_done()
        
if __name__ == "__main__":
    num_worker_threads = 5
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(i,))
        t.start()
        threads.append(t)
        
    for item in range(20):
        q.put(item)
        
    # 모든 작업이 끝날 때까지 대기한다(block).
    q.join()
    
    #워커 스레드를 종료한다(stop).
    for i in range(num_worker_threads):
        q.put(None)
    for t in threads:
        t.join()

## 6.1.3 뮤텍스와 세마포어

### 뮤텍스
- 공유 리소스에 한번에 하나의 스레드만 접근 할 수 있도록 하는 상호 배제(mutual exclusion) 동시성 제어 정책을 강제하기 위해 설계
- 예를 들어 한 스레드가 배열을 수정하고 있다고 가정. 배열 작업을 절반 이상 수행했을 때, 프로세서가 다른 스레드로 전환한다면 두 스레드가 동시에 배열을 수정하는 일이 발생함
- 개념적으로 뮤텍스는 1부터 시작하는 정수
- 스레드는 배열을 변경해야 할 때마다 뮤텍스를 '잠근다'. 즉, 스레드는 뮤텍스가 양수가 될 때까지 대기한 다음 숫자를 1 감소시킨다(이것이 곧 락). 배열 수정을 마치면 뮤텍스가 잠금 해제되어 숫자가 1 증가(언락)
- 배열 수정 전 뮤텍스를 잠근 후, 수정 작업이 끝나고 잠금을 해제하면, 두 스레드가 배열을 동시에 수정하는 일은 일어나지 않음

참조: 2_threading_mutex.py

### 세마포어(semaphore)
- 뮤텍스보다 더 일반적으로 사용되는 개념
- 세마포어는 1보다 큰 수로 시작 가능
- 세마포어 값은 곧 한번에 자원에 접근할 수 있는 스레드의 수
- 세마포어는 뮤텍스의 락 및 언락 작업과 유사한 대기(wait) 및 신호(signal) 작업을 지원
- 파이썬의 뮤텍스(락)과 세마포어에 관한 내용은 threading 모듈의 공식 문서를 참조

참조: 3_threading_semaphore.py